In [ ]:
%matplotlib inline

import gym
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import Video

from video_recorder import VideoRecorder

env_name = "CartPole-v1"
record = True
video_name = "test_0"

env = gym.make(env_name)
state = env.reset()
done = False
total_reward = 0


def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode="rgb_array"))
    plt.axis("off")
    display.clear_output(wait=True)
    display.display(plt.gcf())


if record:
    video_recorder = VideoRecorder(env, f"videos/{video_name}.mp4")

states = 0
while not done:
    states += 1
    if record:
        video_recorder.capture_frame()
    else:
        show_state(env)
    state, reward, done, _ = env.step(env.action_space.sample())  # Take a random action
    total_reward += reward

print(f"total reward: {total_reward}")
if record:
    video_recorder.close()

total reward: 19.0


In [ ]:
# Display a saved video
Video("videos/test_0.mp4")